# Exploration of execution

Execution 

```{bash}
cd /home/ec2-user/SageMaker/foodi-ml
source activate python3
export DATA_PATH=/home/ec2-user/SageMaker/data/
python run.py options/adapt/foodi-ml/i2t.yaml

#watch -n 1 "nvidia-smi"
```

In [1]:
import os
os.chdir("/home/ec2-user/SageMaker/foodi-ml/")

In [2]:
#!pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl

In [3]:
#!pip install -r requirements.txt

In [4]:
import os
import torch
from tqdm import tqdm
from addict import Dict

import params
from retrieval.train import train
from retrieval.utils import helper
from retrieval.model import loss
from retrieval.model.model import Retrieval
from retrieval.data.loaders import get_loaders
from retrieval.utils.logger import create_logger
from retrieval.utils.helper import load_model
from retrieval.utils.file_utils import load_yaml_opts, parse_loader_name

# Functions

In [5]:
def get_data_path(opt):
    if 'DATA_PATH' not in os.environ:
        if not opt.dataset.data_path:
            raise Exception('''
                DATA_PATH not specified.
                Please, run "$ export DATA_PATH=/path/to/dataset"
                or add path to yaml file
            ''')
        return opt.dataset.data_path
    else:
        return os.environ['DATA_PATH']

In [6]:
def get_tokenizers(train_loader):
    tokenizers = train_loader.dataset.tokenizer
    if type(tokenizers) != list:
        tokenizers = [tokenizers]
    return tokenizers

In [7]:
def set_criterion(opt, model):
    if 'name' in opt.criterion:
        logger.info(opt.criterion)
        multimodal_criterion = loss.get_loss(**opt.criterion)
        multilanguage_criterion = loss.get_loss(**opt.criterion)
    else:
        multimodal_criterion = loss.ContrastiveLoss(**opt.criterion)
        multilanguage_criterion = loss.ContrastiveLoss(**opt.ml_criterion)
    set_model_criterion(opt, model, multilanguage_criterion, multimodal_criterion)
    # return multimodal_criterion, multilanguage_criterion


def set_model_criterion(opt, model, multilanguage_criterion, multimodal_criterion):
    model.mm_criterion = multimodal_criterion
    model.ml_criterion = None
    if len(opt.dataset.adapt.data) > 0:
        model.ml_criterion = multilanguage_criterion

# Run

In [8]:
os.environ["DATA_PATH"] = "/home/ec2-user/SageMaker/dataset/"

In [9]:
options = "options/adapt/foodi-ml/i2t.yaml"

In [10]:
args = {
    "options": options,
}
args = Dict(args)
opt = load_yaml_opts(args.options)
opt['operation_name']='test'

In [11]:
logger = create_logger(level='debug' if opt.engine.debug else 'info')
#logger.info(f'Used args   : \n{args}')
#logger.info(f'Used options: \n{opt}')

In [12]:
# Get path of the data
data_path = get_data_path(opt)

In [13]:
# Get loaders
train_loader, val_loaders, adapt_loaders = get_loaders(data_path, args.local_rank, opt)

2021-09-27 15:17:49,599 - [INFO    ] - Loaded vocab containing 4963 tokens
2021-09-27 15:17:49,599 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-09-27 15:17:49,600 - [INFO    ] - Created tokenizer with init 4963 tokens.


self._data_path:  /home/ec2-user/SageMaker/dataset
foodiml
self.full_path:  /home/ec2-user/SageMaker/dataset/foodiml


2021-09-27 15:17:50,467 - [INFO    ] - [FoodiML] Loaded 7000 images annotated 
2021-09-27 15:17:50,473 - [INFO    ] - Loaded vocab containing 4963 tokens
2021-09-27 15:17:50,473 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-09-27 15:17:50,474 - [INFO    ] - Created tokenizer with init 4963 tokens.
2021-09-27 15:17:50,595 - [INFO    ] - [FoodiML] Loaded 1500 images annotated 
2021-09-27 15:17:50,596 - [INFO    ] - Adapt loaders: 0


self._data_path:  /home/ec2-user/SageMaker/dataset
foodiml
self.full_path:  /home/ec2-user/SageMaker/dataset/foodiml


In [14]:
tokenizers = get_tokenizers(train_loader)

In [15]:
%%time
model = Retrieval(**opt.model, tokenizers=tokenizers)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/ec2-user/.cache/torch/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:05<00:00, 46880215.40it/s]
2021-09-27 15:18:21,251 - [INFO    ] - Image encoder created: ('full_image',)
2021-09-27 15:18:21,537 - [INFO    ] - Text encoder created: gru_glove
2021-09-27 15:18:21,667 - [INFO    ] - Created similarity: AdaptiveEmbeddingI2T(
  (norm): Normalization(
    (norm): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (adapt_txt): ADAPT(
    (fc_gamma): Sequential(
      (0): Linear(in_features=2048, out_features=2048, bias=True)
    )
    (fc_beta): Sequential(
      (0): Linear(in_features=2048, out_features=2048, bias=True)
    )
  )
  (fovea): Fovea(smooth=10,train_smooth: False)
)
2021-09-27 15:18:21,668 - [INFO    ] - Created similarity: AdaptiveEmbeddingI2T_eval(
  (norm): Normalization(
    (norm): BatchNorm1d(2048, eps=1e-05, momentum=0.

new similarity class initialised


2021-09-27 15:18:26,558 - [INFO    ] - Setting devices: img: cuda,txt: cuda, loss: cuda
2021-09-27 15:18:26,558 - [INFO    ] - Using similarity: ('adapt_i2t',)


CPU times: user 4.35 s, sys: 1.82 s, total: 6.17 s
Wall time: 12 s


In [16]:
print_fn = (lambda x: x) if not model.master else tqdm.write

In [17]:
set_criterion(opt, model)

In [18]:
# Trainer
trainer = train.Trainer(
        model=model,
        args=opt,
        sysoutlog=print_fn,
        path=opt.exp.outpath,
        world_size=1 # TODO
)

In [19]:
trainer.setup_optim(
        lr=opt.optimizer.lr,
        lr_scheduler=opt.optimizer.lr_scheduler,
        clip_grad=opt.optimizer.grad_clip,
        log_grad_norm=False,
        log_histograms=False,
        optimizer=opt.optimizer,
        freeze_modules=opt.model.freeze_modules
)

2021-09-27 15:18:35,698 - [INFO    ] - lr 0.001
2021-09-27 15:18:35,698 - [INFO    ] - [0.5, 2.0, 4000]
2021-09-27 15:18:35,699 - [INFO    ] - [10000, 20000, 3000]


Freezing model.txt_enc.embed.glove
lr: 0.001, #layers: 478, #params: 100,588,612
Total Params: 102,077,512, 


In [20]:
print(train_loader)

In [24]:
opt.engine.valid_interval

31

In [23]:
trainer.fit(
        train_loader=train_loader,
        valid_loaders=val_loaders,
        lang_loaders=adapt_loaders,
        nb_epochs=opt.engine.nb_epochs,
        valid_interval=opt.engine.valid_interval,
        log_interval=opt.engine.print_freq
)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
Epochs:   0%|          | 0/10 [00:10<?, ?it/s]

Batch_Time:   0.883552, Countdown:  50.000000, Epoch:   0.000000, Iteration:  10.000000, K:   0.086441, Loss: 286.250061, Lr_Base:   0.000504, Norm: 647.066952, Total_Loss: 286.250061


Epochs:   0%|          | 0/10 [00:20<?, ?it/s]

Batch_Time:   1.167607, Countdown:  50.000000, Epoch:   0.000000, Iteration:  20.000000, K:   0.165410, Loss: 291.062805, Lr_Base:   0.000508, Norm: 715.587118, Total_Loss: 291.062805


Epochs:   0%|          | 0/10 [00:30<?, ?it/s]

Batch_Time:   0.938989, Countdown:  50.000000, Epoch:   0.000000, Iteration:  30.000000, K:   0.237553, Loss: 233.167374, Lr_Base:   0.000511, Norm: 513.135821, Total_Loss: 233.167374


Pred  :   0%|          | 0/1500 [00:00<?, ?it/s]

nb_loaders:  1
Evaluating  1/ 1 - foodiml.dev.es
evaluation.predict_loader begins
Evaluation begins


after doing forward in one batch in evaluation: img_emb.size(): torch.Size([1, 2048, 49])
after doing forward in one batch in evaluation: cap_emb.size(): torch.Size([1, 27, 2048])
Trying to allocate a tensor in CPU of (1500, 2048, 49)
New size of the img_embs = (1500, 2048)
Declaring giant matrix for captions...
Declared
dimension of image vector:  torch.Size([1, 2048])
dimension of cap_emb:  torch.Size([1, 27, 2048])


RuntimeError: The size of tensor a (27) must match the size of tensor b (2048) at non-singleton dimension 1

# EXPLORE

In [ ]:
torch.__version__

In [ ]:
#!pip install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-win_amd64.whl

In [ ]:
import torch
a = torch.randn(3, 3, device="cuda:0")
a = torch.mm(a, a.t()) # make symmetric positive-definite
torch.cholesky(a)

## Data Loaders

In [ ]:
index = 0
image_id = train_loader.dataset.data_wrapper.image_ids[index]
caption = train_loader.dataset.captions[index]
cap_tokens = train_loader.dataset.tokenizer(caption)
image = train_loader.dataset.load_img(image_id)

import matplotlib.pyplot as plt
plt.imshow(image.permute(1, 2, 0)  )

In [ ]:
import numpy as np
np.max(list(train_loader.dataset.data_wrapper.image_ids))

In [ ]:
len(train_loader.dataset.captions)

In [ ]:
image.shape

In [76]:
# Check valid loader
#image_id2 = val_loaders[0].dataset.data_wrapper.image_ids[index]
#image2 = val_loaders[0].dataset.load_img(image_id2)
#import matplotlib.pyplot as plt
#plt.imshow(  image2.permute(1, 2, 0)  )